In [0]:
# -*- coding: utf-8 -*-

"""
Created on Fri Mar  8 07:39:17 2019

@author: james
"""

import bs4
import requests
import pandas as pd
import numpy as np
from google.colab import files

###############################################################################
#    Section 1 - The stem link
###############################################################################

#********************************* FILL IN ************************************
OGA = "https://www.ogauthority.co.uk/lessons-learned/"
#******************************************************************************

link = (OGA + "index.html")
data = requests.get(link)
soup = bs4.BeautifulSoup(data.content, 'html.parser')

###############################################################################
#    Section 2 - Getting section URLs
###############################################################################

#********************************* FILL IN ************************************
section = soup.find_all("a")
#******************************************************************************

links = []

for item in section[1:][:6]:
    alpha=str(item).split("href=")[1].split("><")[0][1:][:-1]
    
    # Passing each section URL to bs4
    link = (OGA + alpha)
    
    print([link])
    
    data = requests.get(link)
    soup = bs4.BeautifulSoup(data.content, 'html.parser')
    
    subsec = []
    
    ###############################################################################
    #    Section 3 - Getting sub-section URLs
    ###############################################################################
    
    for word in ["planning","execution","cost"]:
        x=str(soup.find_all("div",{"class":word})[0]).split("href=")[1].split(">")[0]
        if "/" in x:
            x2=x.split("/")[-1][:-1]
        else:
            x2=x[1:][:-1]
    
        subsec.append(x2) 
    
    print(subsec)
    
    ###############################################################################
    #    Section 4 - Getting sub-sub-section URLs
    ###############################################################################
    
    k=0
    for word in ["planning","execution","cost"]:
        link = OGA+link.split("/")[-3]+"/"+word+"/"+subsec[k]
        
        data = requests.get(link)
        soup = bs4.BeautifulSoup(data.content, 'html.parser')
        
        #********************************* FILL IN ************************************
        # Getting the sub-sub-sections
        subsubsections = soup.find_all("a")
        #******************************************************************************
        
        k+=1
            
        for item in subsubsections:
            # Split on "href" and "><"
            beta=str(item).split("href=")[1].split("><")[0]
            # filtering out links we don't want
            if ".." not in beta and ">" not in beta:
                url = str('/'.join(str(link).split("/")[:-1])+'/'+beta[1:-1])
                links.append(url)
                
                print(url)  

###############################################################################
#    Section 5 - Creating a unique list of all the urls
###############################################################################
         
#Prints the numer of unique links
links = list(dict.fromkeys(links))
print(len(links))

all_text = []
sections = []
subsections = []
subsubsections = []

for x in range(len(links)):
    link = links[x]
    data = requests.get(link)
    soup = bs4.BeautifulSoup(data.content, 'html.parser')
    a = len(str(soup.find_all("p")).split("</p>"))-1
    
    ###############################################################################
    #   Section 6 - Getting the sub-sub-sub-section text. But not the bullet points
    ###############################################################################
    
    for i in range(0,a):
        text = str(soup.find_all("p")[i]).split(">")[2].split("<")[0]
        all_text.append(text)
        section = ' '.join(word for word in str(link).split("/")[-1].split("-")[:-1])
        sections.append(section)
        subsection = str(link).split("/")[-2]
        subsections.append(subsection)
        subsubsection = str(link).split("-")[-1].split(".")[0]
        subsubsections.append(subsubsection)

###############################################################################
#    Section 7 - Creating a data frame from all our informationn
###############################################################################            
            
# Creating a dataframe of the scraped text
df = pd.DataFrame({'Section':np.array(sections),
                   'Subsection':np.array(subsections),
                   'Sub-Subsection':np.array(subsubsections),
                   'Text':np.array(all_text)})

###############################################################################
#    Section 8 - Remapping the shorthand words
###############################################################################     
    
# Remapping the 'subsection' collumn
def remap(x):
    if x == 'pm':
        return 'Project Management'
    if x == 'froc':
        return 'Facilities Running/ Owner Costs'
    if x == 'wa':
        return 'Well Abandonment'
    if x == 'fpms':
        return 'Facilities/ Pipeline Making Safe'
    if x == 'si':
        return 'Subsea Infrastructure'
    if x == 'tp':
        return 'Topsides Preparation'
    if x == 'tr':
        return 'Topsides Removal'
    if x == 'sr':
        return 'Substructure Removal'
    if x == 'tsor':
        return 'Topside & Substructure Onshore Recycling'
    if x == 'srn':
        return 'Site Remediation'
    if x == 'm':
        return 'Monitoring'
    else:
        return 'na'
    

 
df['Sub-Subsection'] = df['Sub-Subsection'].apply(remap)

###############################################################################
#    Section 9 - Creating our CSV file and downloading it
###############################################################################  


#********************************* FILL IN ************************************
df.to_csv("OGA_lessons_learned.csv")
files.download("OGA_lessons_learned.csv")
#******************************************************************************

['https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-pm.html']
['operating-pm.html', 'operating-wa.html', 'operating-pm.html']
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-pm.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-froc.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-wa.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-fpms.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-tr.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-sr.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-tsor.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-srn.html
https://www.ogauthority.co.uk/lessons-learned/1/planning/operating-m.html
https://www.ogauthority.co.uk/lessons-learned/1/execution/operating-wa.html
https://www.ogauthority.co.uk/lessons-learned/1/execution/operating-fpms.html
https://www.ogauthorit